In [3]:
import pandas as pd
import os
import csv

In [26]:
outlets = pd.read_csv("/shared/3/projects/benlitterer/localNews/NELAdata/CSVs/outlets.csv")
articles = pd.read_csv("/shared/3/projects/benlitterer/localNews/NELAdata/CSVs/articles.csv")
demographics = pd.read_csv("/shared/3/projects/benlitterer/localNews/NELAdata/CSVs/demographics.csv")
risks = pd.read_csv("/shared/3/projects/benlitterer/localNews/NELAdata/CSVs/risks.csv")

In [28]:
outletDem = pd.merge(outlets, demographics, how = "left", on="fips")

In [29]:
#merge data into main local news dataframe 
outletDf = pd.merge(outletDem, risks, how="left", on="fips")
localDf = pd.merge(articles, outletDf, how="left", on="sourcedomain_id")

In [30]:
#get nela 2022
nelaGTLocation = "/shared/3/projects/benlitterer/localNews/NELA-gt-2020/nela-gt-2020/newsdata"

outletDfList = []
i = 0 
for outletName in os.listdir(nelaGTLocation): 
    if i % 50 == 0: 
        print(i)
    i+= 1 
    
    #get dataframe for this outlet, add to list 
    outletPath = nelaGTLocation + "/" + outletName
    outletDf = pd.read_json(outletPath)
    outletDfList.append(outletDf)

#resetting the index prevents us from getting a weird nested index later 
gtDf = pd.concat(outletDfList).reset_index(drop=True)
gtDf = gtDf.rename({"id":"gt_id"})
localDf = localDf.rename(columns={"url_x":"url", "article_id":"local_id"}).drop(columns=["url_y"])

0
50
100
150
200
250
300
350
400
450
500


In [38]:
localDf.columns

Index(['local_id', 'sourcedomain_id', 'date', 'title', 'content', 'url',
       'fips', 'source', 'description', 'onlinesince', 'rank', 'state', 'city',
       'lon', 'lat', 'county', 'total_population', 'white_pct', 'black_pct',
       'hispanic_pct', 'nonwhite_pct', 'foreignborn_pct', 'female_pct',
       'age29andunder_pct', 'age65andolder_pct', 'median_hh_inc',
       'clf_unemploy_pct', 'lesshs_pct', 'lesscollege_pct',
       'lesshs_whites_pct', 'lesscollege_whites_pct', 'rural_pct',
       'ruralurban_cc', 'predrt_0', 'predrt_12', 'predrt_3'],
      dtype='object')

In [41]:
len(localDf) - len(localDf.drop_duplicates(["url"]))

86264

In [42]:
len(localDf) - len(localDf.drop_duplicates(["url", "content"]))

71030

In [43]:
len(localDf) - len(localDf.drop_duplicates(["source", "title"]))

56349

In [44]:
#merge national and local data
#create 'national' column 
localDf["national"] = [False for i in range(0, len(localDf))]
gtDf["national"] = [True for i in range(0, len(gtDf))]

#merge local and national-2020 news together 
totalDf = pd.concat([localDf, gtDf]).reset_index(drop=True)


In [47]:
len(localDf)

1445509

In [48]:
len(gtDf)

1779127

In [3]:
"""
#shortcut since an earlier script wrote this data here
previousDf = pd.read_csv("/shared/3/projects/benlitterer/localNews/MVP1/mergedArticles.tsv", sep="\t")
totalDf = previousDf
"""

/opt/anaconda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (1,2,6,9,10,11,12,13,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,38,39,40,41) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [50]:
#get NELA-2021
nelaGTLocation = "/shared/3/projects/benlitterer/localNews/NELA-gt-2021/json/nela-gt-2021/newsdata"

outletDfList = []
i = 0 
for outletName in os.listdir(nelaGTLocation): 
    if i % 50 == 0: 
        print(i)
    i+= 1 
    
    #get dataframe for this outlet, add to list 
    outletPath = nelaGTLocation + "/" + outletName
    outletDf = pd.read_json(outletPath)
    outletDfList.append(outletDf)

#resetting the index prevents us from getting a weird nested index later 
gtDf = pd.concat(outletDfList).reset_index(drop=True)

#create 'national' column 
gtDf["national"] = [True for i in range(0, len(gtDf))]

0
50
100
150
200
250
300
350


In [51]:
len(gtDf)

1856509

In [52]:
#concatenate local, nela 2020 with nela 2021
mergedDf = pd.concat([totalDf, gtDf]).reset_index(drop=True)

In [53]:
len(mergedDf)

5081145

In [24]:
sum(gtDf["url"].isna())

0

# look into potential keys for NER

In [58]:
# data questions: 
# 1053 na titles, no na sources 
#len(mergedDf.drop_duplicates(["title","source"]))

#311152 duplicates with title + source key 
#len(mergedDf) - len(mergedDf.drop_duplicates(["title", "source"]))

#194287 duplicates with title + source + content 
#so 2/3 of those are also duplicates with text
#len(mergedDf) - len(mergedDf.drop_duplicates(["title", "source", "content"]))

#435689 duplicate urls 
#len(mergedDf) - len(mergedDf.drop_duplicates(["url"]))

#288445 duplicates with url + content
#so about half of those have different content 
#len(mergedDf) - len(mergedDf.drop_duplicates(["url", "content"]))


288445

# Other data notes:

In [62]:
#we don't have any n/a sources which is good 

#5,081,145 is the correct number of articles to have according to the papers 

In [63]:
mergedDf.columns

Index(['local_id', 'sourcedomain_id', 'date', 'title', 'content', 'url',
       'fips', 'source', 'description', 'onlinesince', 'rank', 'state', 'city',
       'lon', 'lat', 'county', 'total_population', 'white_pct', 'black_pct',
       'hispanic_pct', 'nonwhite_pct', 'foreignborn_pct', 'female_pct',
       'age29andunder_pct', 'age65andolder_pct', 'median_hh_inc',
       'clf_unemploy_pct', 'lesshs_pct', 'lesscollege_pct',
       'lesshs_whites_pct', 'lesscollege_whites_pct', 'rural_pct',
       'ruralurban_cc', 'predrt_0', 'predrt_12', 'predrt_3', 'national', 'id',
       'author', 'published', 'published_utc', 'collection_utc'],
      dtype='object')

In [64]:
#make sure we have a completely unique index 
mergedDf = mergedDf.reset_index(drop=True)

#get a new index to use as a key 
mergedDf = mergedDf.reset_index()
mergedDf = mergedDf.rename(columns={"index":"key"})

#sanity check to make sure key is unique 
len(mergedDf) - len(mergedDf.drop_duplicates(["key"]))

In [10]:
mergedDf["content"] = mergedDf["content"].fillna("")

In [11]:
#write news data to output file 
mergedDf.to_csv("/shared/3/projects/newsDiffusion/data/processed/newsData/fullMergedNELAdata.tsv", sep="\t", quoting=csv.QUOTE_NONNUMERIC)

In [14]:
df = pd.read_csv("/shared/3/projects/newsDiffusion/data/interim/NEREmbedding/NERSplitsComplete/NERSplits0topics.tsv", sep="\t", header=None)

In [17]:
df.tail(20)

,0,1
1137,1137,"[('CARDINAL', 'one'), ('WORK_OF_ART', 'For Lov..."
1138,1138,"[('PRODUCT', 'Frodo'), ('ORG', 'Gandalf'), ('T..."
1139,1139,"[('PERSON', 'Clark'), ('DATE', 'March 23-28, 2..."
1140,1140,"[('PERSON', 'Andy Beshear'), ('ORG', 'Healthy ..."
1141,1141,"[('PERSON', 'aka potluck'), ('CARDINAL', 'one'..."
1142,1142,"[('ORG', 'the Louisiana State Police Special V..."
1143,1143,"[('GPE', 'NATCHITOCHES'), ('PERSON', 'Missy Bi..."
1144,1144,"[('ORG', 'Bossier Parish School'), ('DATE', 't..."
1145,1145,"[('ORG', 'LHSAA'), ('DATE', 'Friday'), ('TIME'..."
1146,1146,"[('DATE', 'April 12, 2020'), ('ORG', 'The Kans..."
